# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherDF = pd.read_csv("outputData/WeatherData.csv")
weatherDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Qaanaaq,77.4840,-69.3632,40.55,92,100,0.38,GL,1658709538
1,Souillac,-20.5167,57.5167,66.60,84,38,7.07,MU,1658709538
2,Ushuaia,-54.8000,-68.3000,35.26,64,40,5.75,AR,1658709528
3,Adrar,20.5022,-10.0711,95.61,7,5,10.13,MR,1658709539
4,Yārāda,17.6500,83.2667,77.32,100,40,2.30,IN,1658709539


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# create coordinates by joining lat and long
locations = weatherDF[["Lat", "Lng"]]

# Humidity as the weights
humidity = weatherDF["Humidity"].astype(float)

# Plotting the heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1.3)

# Create heat layer
heatLayer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 100)

# Add layer
fig.add_layer(heatLayer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
idealWeatherDF = weatherDF.loc[(weatherDF["Max Temp"] < 80) & (weatherDF["Max Temp"] > 70) & (weatherDF["Wind Speed"] < 10) & (weatherDF["Cloudiness"] == 0) & (weatherDF["Max Temp"] > 70) & (weatherDF["Wind Speed"] < 10) & (weatherDF["Humidity"] < 80)]
len(idealWeatherDF)

8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotelDF = idealWeatherDF
hotelDF["Hotel Name"] = ""
hotelDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
254,Broome,42.2506,-75.8330,75.60,78,0,8.05,US,1658709172,
284,Solnechnyy,50.7214,136.6319,79.21,55,0,4.90,RU,1658709213,
366,Zalţan,32.9470,11.8665,77.58,67,0,5.64,LY,1658709330,
381,Nobres,-14.7203,-56.3275,77.72,34,0,4.47,BR,1658709352,
408,Fairbanks,64.8378,-147.7164,74.08,33,0,6.91,US,1658709394,


In [6]:
# search for hotels nearby
# dictionary of parameters to search for hotels nearby
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}

# Loop through the hotel dataframe and run a lat/long search
for index, row in hotelDF.iterrows():

    # create lat and long variable
    lat = row["Lat"]
    lng = row["Lng"]

    # Update parameters dictionary
    parameters["location"] = f"{lat},{lng}"
    
    # Define baseURL
    baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make API request
    response = requests.get(baseURL, params=parameters).json()
    
    # Add data to DF
    try:
        hotelDF.loc[index,"Hotel Name"] = response["results"][0]["name"]
    
    except:
        print(f"Unable to find hotel near {hotelDF.loc[index, 'City']}")

In [7]:
hotelDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
254,Broome,42.2506,-75.8330,75.60,78,0,8.05,US,1658709172,Chenango Valley State Park
284,Solnechnyy,50.7214,136.6319,79.21,55,0,4.90,RU,1658709213,Hotel complex Pastoral
366,Zalţan,32.9470,11.8665,77.58,67,0,5.64,LY,1658709330,Fars Alfelale
381,Nobres,-14.7203,-56.3275,77.72,34,0,4.47,BR,1658709352,Hotel Portal Nobres
408,Fairbanks,64.8378,-147.7164,74.08,33,0,6.91,US,1658709394,Westmark Fairbanks Hotel & Conference Center


In [8]:
# adding a clickable tool tip to the marker
infoBox = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotelInfo = [infoBox.format(**row) for index, row in hotelDF.iterrows()]

# Store the locations
coordinates = hotelDF[["Lat", "Lng"]]

# Add a marker layer on the heatmap 
markerLayer = gmaps.marker_layer(coordinates, info_box_content=hotelInfo)

# Add marker layers
fig.add_layer(markerLayer)

In [10]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))